## SQL X Pandas: Duas abordagens para uma mesma consulta

### Bibliotecas utilizadas

In [1]:
import pandas as pd
import sqlite3

### Conexão com a base de dados

In [2]:
conn = sqlite3.connect('../data/database.db')
c = conn.cursor()

### Carregamento dos dados

Lojas

In [3]:
command = '''SELECT * FROM lojas'''

c.execute(command)

columns = [column[0] for column in c.description]

lojas = pd.DataFrame(c.fetchall(), columns=columns)
lojas.head()

,idLoja,descCidadeLoja,descEstadoLoja
0,1,Florianópolis,SC
1,2,Rio de Janeiro,RJ
2,3,Porto Alegre,RS
3,4,Curitiba,PR
4,5,São Paulo,SP


Produtos

In [4]:
command = '''SELECT * FROM produtos'''

c.execute(command)

columns = [column[0] for column in c.description]

produtos = pd.DataFrame(c.fetchall(), columns=columns)
produtos.head()

,idProduto,descNomeProduto,descTamanhoProduto
0,1,Calcinha,P
1,2,Calcinha,M
2,3,Calcinha,G
3,4,Camiseta,P
4,5,Camiseta,M


Clientes

In [5]:
command = '''SELECT * FROM clientes'''

c.execute(command)

columns = [column[0] for column in c.description]

clientes = pd.DataFrame(c.fetchall(), columns=columns)
clientes.head()

,idCliente,descCidadeCliente,descEstadoCliente,dtNascimento,descSexo
0,1,Curitiba,PR,1985-06-28 00:00:00,Masculino
1,2,Florianópolis,SC,1987-01-10 00:00:00,Masculino
2,3,Rio de Janeiro,RJ,1979-11-05 00:00:00,Masculino
3,4,Rio de Janeiro,RJ,1983-11-25 00:00:00,Feminino
4,5,Florianópolis,SC,1969-04-07 00:00:00,Masculino


Vendas

In [6]:
command = '''SELECT * FROM vendas'''

c.execute(command)

columns = [column[0] for column in c.description]

vendas = pd.DataFrame(c.fetchall(), columns=columns)
vendas['dtVenda'] = pd.to_datetime(vendas['dtVenda'])
vendas.head()

,idVenda,idProduto,idCliente,vlDesconto,vlPrecoUnitario,qtProduto,idLoja,dtVenda
0,1,48,1,0.08,249.2,1,4,2018-12-10
1,2,27,1,0.10,162.4,1,4,2019-04-25
2,3,1,1,0.10,194.6,1,4,2019-07-17
3,4,30,2,0.10,201.6,2,1,2018-04-10
4,5,36,2,0.10,406.0,1,1,2019-01-21


### Consultas

#### Pergunta 1
Qual a quantidade de produtos únicos que foram vendidos?

In [7]:
len(vendas['idProduto'].unique())

73

#### Pergunta 2
Quais os três produtos mais vendidos e as quantidades destes?

In [8]:
join = pd.merge(vendas, produtos, on='idProduto', how='left')

colunas = ['idProduto', 'descNomeProduto', 'descTamanhoProduto']
groupby = join.groupby(colunas, as_index=False).agg(somaQtProduto=('qtProduto', 'sum'))

orderby = groupby.sort_values(by='somaQtProduto', ascending=False)

orderby.head(3)

,idProduto,descNomeProduto,descTamanhoProduto,somaQtProduto
62,63,Casaco,G,1020
59,60,Camisa,G,1018
8,9,Blusa,G,998


#### Pergunta 3
Quais as quantidades mensais vendidas do produto camisa de tamanho G no ano de 2019?

In [9]:
select = vendas.assign(dtMesVenda=vendas['dtVenda'].dt.month)

left_join = pd.merge(select, produtos, on='idProduto', how='left')

where = left_join[(left_join['dtVenda'].dt.year == 2019) & \
                  (left_join['descNomeProduto'] == 'Camisa') & \
                  (left_join['descTamanhoProduto'] == 'G')]


groupby = where.groupby('dtMesVenda', as_index=False).agg(somaQtProduto=('qtProduto', 'sum'))
groupby.head(12)

,dtMesVenda,somaQtProduto
0,1,30
1,2,22
2,3,27
3,4,31
4,5,35
5,6,56
6,7,38
7,8,65
8,9,25
9,10,30


#### Pergunta 4
Qual a média da receita líquida mensal de vendas no ano de 2019?

In [10]:
select = vendas.assign(mesVenda = vendas['dtVenda'].dt.month,
                       vlPreco = vendas['vlPrecoUnitario']*vendas['qtProduto'])

where = select[(select['dtVenda'].dt.year == 2019)]

groupby = where.groupby('mesVenda', as_index=False).agg(vlSomaReceitaLiquida=('vlPreco', 'sum'))

groupby['vlSomaReceitaLiquida'].mean()

324240.9333333333

#### Pergunta 5
Qual loja teve a maior receita líquida de vendas no ano de 2019?

In [11]:
select = vendas.assign(vlPreco = vendas['vlPrecoUnitario']*vendas['qtProduto'])

left_join = pd.merge(select, lojas, on='idLoja', how='left')

where = left_join[(left_join['dtVenda'].dt.year == 2019)]

colunas = ['idLoja', 'descCidadeLoja']
groupby = where.groupby(colunas, as_index=False).agg(vlSomaReceitaLiquida=('vlPreco', 'sum'))

orderby = groupby.sort_values(by='vlSomaReceitaLiquida', ascending=False)
orderby.head(3)

,idLoja,descCidadeLoja,vlSomaReceitaLiquida
0,1,Florianópolis,992059.6
1,2,Rio de Janeiro,797032.6
2,3,Porto Alegre,736108.8
